In [1]:
import sys
sys.version

'3.6.5 (v3.6.5:f59c0932b4, Mar 28 2018, 17:00:18) [MSC v.1900 64 bit (AMD64)]'

In [2]:
import platform
platform.python_version()

'3.6.5'

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urlencode
import urllib3
import string
import json

#### https://csdl-api.computer.org/api/v1/graphql (GraphiQL 주소)

##### Periodicals로 idPrefix, title, yearFrom, yearTo 얻기

In [4]:
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                      json={
                          "query":'query { periodicals(pubType:"journal") { idPrefix title yearFrom yearTo } }'
                      })
body = resp.json()

In [5]:
print(json.dumps(body, indent=4))

{
    "data": {
        "periodicals": [
            {
                "idPrefix": "cq",
                "title": "Colloquium",
                "yearFrom": 2017,
                "yearTo": 2019
            },
            {
                "idPrefix": "dc",
                "title": "IEEE Journal on Exploratory Solid-State Computational Devices and Circuits",
                "yearFrom": 2015,
                "yearTo": 2019
            },
            {
                "idPrefix": "ta",
                "title": "IEEE Transactions on Affective Computing",
                "yearFrom": 2010,
                "yearTo": 2019
            },
            {
                "idPrefix": "bd",
                "title": "IEEE Transactions on Big Data",
                "yearFrom": 2015,
                "yearTo": 2019
            },
            {
                "idPrefix": "cc",
                "title": "IEEE Transactions on Cloud Computing",
                "yearFrom": 2013,
                "yearTo": 2019


In [6]:
journals = [] # (title, idPrefix, yearFrom, yearTo)

for i in range(0,len(body["data"]["periodicals"])):
    journals.append( ( (body["data"]["periodicals"][i]['title']), body["data"]["periodicals"][i]['idPrefix'], body["data"]["periodicals"][i]['yearFrom'], body["data"]["periodicals"][i]['yearTo']) )

In [7]:
journals

[('Colloquium', 'cq', 2017, 2019),
 ('IEEE Journal on Exploratory Solid-State Computational Devices and Circuits',
  'dc',
  2015,
  2019),
 ('IEEE Transactions on Affective Computing', 'ta', 2010, 2019),
 ('IEEE Transactions on Big Data', 'bd', 2015, 2019),
 ('IEEE Transactions on Cloud Computing', 'cc', 2013, 2019),
 ('IEEE Transactions on Computational Intelligence and AI in Games',
  'ci',
  2009,
  2017),
 ('IEEE Transactions on Computers', 'tc', 1968, 2019),
 ('IEEE Transactions on Dependable and Secure Computing', 'tq', 2004, 2019),
 ('IEEE Transactions on Emerging Topics in Computing', 'ec', 2013, 2019),
 ('IEEE Transactions on Haptics', 'th', 2008, 2017),
 ('IEEE Transactions on Knowledge & Data Engineering', 'tk', 1989, 2019),
 ('IEEE Transactions on Learning Technologies', 'lt', 2008, 2017),
 ('IEEE Transactions on Mobile Computing', 'tm', 2002, 2019),
 ('IEEE Transactions on Multi-Scale Computing Systems', 'mc', 2015, 2019),
 ('IEEE Transactions on Network Science and Engin

In [8]:
labels = ["journalName", "journalId", "yearFrom", "yearTo"]
journals_meta_df = pd.DataFrame.from_records(journals, columns=labels)

In [9]:
journals_meta_df

,journalName,journalId,yearFrom,yearTo
0,Colloquium,cq,2017,2019
1,IEEE Journal on Exploratory Solid-State Comput...,dc,2015,2019
2,IEEE Transactions on Affective Computing,ta,2010,2019
3,IEEE Transactions on Big Data,bd,2015,2019
4,IEEE Transactions on Cloud Computing,cc,2013,2019
5,IEEE Transactions on Computational Intelligenc...,ci,2009,2017
6,IEEE Transactions on Computers,tc,1968,2019
7,IEEE Transactions on Dependable and Secure Com...,tq,2004,2019
8,IEEE Transactions on Emerging Topics in Computing,ec,2013,2019
9,IEEE Transactions on Haptics,th,2008,2017


##### PeriodicalIssues로 year, volume, issueNum 얻기

In [10]:
#예시
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                      json={
                          "variables":{"idPrefix":"ci"}, #ci는 하나의 예시임. 실제로는 모든 idPrefix 넣으면서 긁어오기
                          "query":'query ($idPrefix : String!) { periodicalIssues(idPrefix : $idPrefix) { label issueNum volume year } }'
                      })
body = resp.json()

In [11]:
print(json.dumps(body, indent=4))

{
    "data": {
        "periodicalIssues": [
            {
                "label": "March",
                "issueNum": "01",
                "volume": "1",
                "year": "2009"
            },
            {
                "label": "June",
                "issueNum": "02",
                "volume": "1",
                "year": "2009"
            },
            {
                "label": "Sept.",
                "issueNum": "03",
                "volume": "1",
                "year": "2009"
            },
            {
                "label": "Dec.",
                "issueNum": "04",
                "volume": "1",
                "year": "2009"
            },
            {
                "label": "March",
                "issueNum": "01",
                "volume": "2",
                "year": "2010"
            },
            {
                "label": "June",
                "issueNum": "02",
                "volume": "2",
                "year": "2010"
            },
   

In [12]:
journals_df = pd.DataFrame()

for i in range(0, len(journals)) :
    resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                          json={
                              "variables":{"idPrefix":"%s"%(journals[i][1])},
                              "query":'query ($idPrefix : String!) { periodicalIssues(idPrefix : $idPrefix) { label issueNum volume year } }'
                          })
    body = resp.json()
    
    labels = ["journalId", "year", "issueNum", "volume"]
    temp_df = pd.DataFrame(body['data']['periodicalIssues'], columns=labels)
    temp_df['journalId'] = journals[i][1]
    journals_df = journals_df.append(temp_df)
    
    
    #Todo : 저널 별로 year, volume, issueNum 저장
    print("Progress : %d of %d (%s)" % (i+1, len(journals), journals[i][1]))

Progress : 1 of 26 (cq)
Progress : 2 of 26 (dc)
Progress : 3 of 26 (ta)
Progress : 4 of 26 (bd)
Progress : 5 of 26 (cc)
Progress : 6 of 26 (ci)
Progress : 7 of 26 (tc)
Progress : 8 of 26 (tq)
Progress : 9 of 26 (ec)
Progress : 10 of 26 (th)
Progress : 11 of 26 (tk)
Progress : 12 of 26 (lt)
Progress : 13 of 26 (tm)
Progress : 14 of 26 (mc)
Progress : 15 of 26 (tn)
Progress : 16 of 26 (td)
Progress : 17 of 26 (tp)
Progress : 18 of 26 (sc)
Progress : 19 of 26 (ts)
Progress : 20 of 26 (su)
Progress : 21 of 26 (si)
Progress : 22 of 26 (tg)
Progress : 23 of 26 (tb)
Progress : 24 of 26 (nt)
Progress : 25 of 26 (ca)
Progress : 26 of 26 (lc)


In [13]:
journals_df

,journalId,year,issueNum,volume
0,cq,2017,01,1
1,cq,2017,02,1
2,cq,2017,03,1
3,cq,2017,04,1
4,cq,2017,05,1
5,cq,2017,06,1
6,cq,2017,07,1
7,cq,2017,08,1
8,cq,2017,09,1
9,cq,2017,10,1


In [14]:
journals_groupbyID = journals_df.groupby('journalId')

In [15]:
journals_groupbyID.get_group('cq')

,journalId,year,issueNum,volume
0,cq,2017,01,1
1,cq,2017,02,1
2,cq,2017,03,1
3,cq,2017,04,1
4,cq,2017,05,1
5,cq,2017,06,1
6,cq,2017,07,1
7,cq,2017,08,1
8,cq,2017,09,1
9,cq,2017,10,1


##### articles과 param (idPrefix, year, issueNum)을 통해 article 크롤링

In [16]:
#예시
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                      json={
                          "variables":{"idPrefix":"ca", "year":"2018", "issueNum":"02"}, #ci, 2018, 02는 하나의 예시임. 실제로는 idPrefix, year, issueNum 바꿔가며 긁어오기
                          "query":'query ($idPrefix : String!, $year : String!, $issueNum : String!) { articles(idPrefix: $idPrefix, year: $year, issueNum: $issueNum) { idPrefix id year pubDate keywords title abstract authors { affiliation fullName givenName surname } } }'
                      })

In [17]:
body = resp.json()

In [18]:
print(json.dumps(body, indent=4))

{
    "data": {
        "articles": [
            {
                "idPrefix": "ca",
                "id": "13rRUwwslBG",
                "year": "2018",
                "pubDate": "2018-07-01",
                "keywords": [
                    "Nonvolatile Memory",
                    "Random Access Memory",
                    "Encryption",
                    "Computer Architecture",
                    "System On Chip",
                    "Oblivious RAM",
                    "Non Volatile Memory",
                    "Memory Security"
                ],
                "title": "LEO: Low Overhead Encryption ORAM for Non-Volatile Memories",
                "abstract": "Data confidentiality attacks utilizing memory access patterns threaten exposure of data in modern main memories. Oblivious RAM\u00a0(ORAM) is an effective cryptographic primitive developed to thwart access-pattern-based attacks in DRAM-based systems. However, in emerging non-volatile memory\u00a0(NVM) systems, the i

In [21]:

def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [22]:
# 모두 한 테이블로 합치는 비효율적 구조. 다른 방법은 없을까? keywords도 어떻게 처리할 지 생각해보자. 
all_articles_df = pd.DataFrame()

for key in log_progress(journals_groupbyID.groups.keys()):
    temp_df = journals_groupbyID.get_group(key)
    
    for _, row in temp_df.iterrows():
        resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                          json={
                              "variables":{"idPrefix":row['journalId'], "year":row['year'], "issueNum":row['issueNum']}, 
                              "query":'query ($idPrefix : String!, $year : String!, $issueNum : String!) { articles(idPrefix: $idPrefix, year: $year, issueNum: $issueNum) { idPrefix id year pubDate keywords title abstract authors { affiliation fullName givenName surname } } }'
                          })
        body = resp.json()
        labels = ['idPrefix', 'id', 'pubDate', 'keywords', 'title', 'abstract']
        all_articles_df = all_articles_df.append(pd.DataFrame.from_records(body['data']['articles'], columns=labels))

In [23]:
all_articles_df

,idPrefix,id,pubDate,keywords,title,abstract
0,bd,13rRUwwslvh,2015-01-01,[],Welcome to the IEEE Transactions on Big Data,Presents an editorial introducting the inaugur...
1,bd,13rRUygT7c8,2015-01-01,[],Introduction to the IEEE Transactions on Big Data,Presents an introduction to the inaugural issu...
2,bd,13rRUwvT9j1,2015-01-01,"[Data Integration, Big Data, Data Mining, Feat...",Methodologies for Cross-Domain Data Fusion: An...,Traditional data mining usually deals with dat...
3,bd,13rRUxBJhxj,2015-01-01,"[Semantics, Internet, Electronic Publishing, S...",Embracing Information Explosion without Chokin...,The explosive popularity of microblogging serv...
0,bd,13rRUxAStUe,2015-07-01,"[Data Models, Computational Modeling, Big Data...",Petuum: A New Platform for Distributed Machine...,What is a systematic way to efficiently apply ...
1,bd,13rRUxASu2L,2015-07-01,"[Distributed Databases, Base Stations, Data Pr...",SMC: A Practical Schema for Privacy-Preserved ...,Data collection is required to be safe and eff...
0,bd,13rRUxAASMi,1900-01-01,[],Guest Editorial: Big Media Data: Understanding...,None
1,bd,13rRUwbJCYJ,1900-01-01,"[Binary Codes, Error Correction Codes, Error C...",Code Consistent Hashing Based on Information-T...,Learning based hashing techniques have attract...
2,bd,13rRUzphDs2,1900-01-01,"[Visualization, Search Engines, Google, Big Da...",Exploration of Image Search Results Quality As...,Image retrieval plays an increasingly importan...
3,bd,13rRUNvyan1,1900-01-01,"[Training, Semantics, Machine Learning, Visual...",Weakly Semi-Supervised Deep Learning for Multi...,"In this paper, we study leveraging both weakly..."


In [24]:
all_articles_df.to_pickle('all_articles_df.p')

In [25]:
all_articles_df = pd.read_pickle('all_articles_df.p')

In [26]:
all_articles_df['pubDate'].iloc[5000]

'1995-04-01'

In [27]:
all_articles_df['pubDate'].str.extract(r'(\d+-\d+-\d+)')

,0
0,2015-01-01
1,2015-01-01
2,2015-01-01
3,2015-01-01
0,2015-07-01
1,2015-07-01
0,1900-01-01
1,1900-01-01
2,1900-01-01
3,1900-01-01


In [28]:
all_articles_df['pubDate'] = pd.to_datetime(all_articles_df['pubDate'], format='%Y-%m-%d', errors='coerce')

In [29]:
all_articles_df = all_articles_df[~ all_articles_df['pubDate'].isnull()]

In [30]:
all_articles_df

,idPrefix,id,pubDate,keywords,title,abstract
0,bd,13rRUwwslvh,2015-01-01,[],Welcome to the IEEE Transactions on Big Data,Presents an editorial introducting the inaugur...
1,bd,13rRUygT7c8,2015-01-01,[],Introduction to the IEEE Transactions on Big Data,Presents an introduction to the inaugural issu...
2,bd,13rRUwvT9j1,2015-01-01,"[Data Integration, Big Data, Data Mining, Feat...",Methodologies for Cross-Domain Data Fusion: An...,Traditional data mining usually deals with dat...
3,bd,13rRUxBJhxj,2015-01-01,"[Semantics, Internet, Electronic Publishing, S...",Embracing Information Explosion without Chokin...,The explosive popularity of microblogging serv...
0,bd,13rRUxAStUe,2015-07-01,"[Data Models, Computational Modeling, Big Data...",Petuum: A New Platform for Distributed Machine...,What is a systematic way to efficiently apply ...
1,bd,13rRUxASu2L,2015-07-01,"[Distributed Databases, Base Stations, Data Pr...",SMC: A Practical Schema for Privacy-Preserved ...,Data collection is required to be safe and eff...
0,bd,13rRUxAASMi,1900-01-01,[],Guest Editorial: Big Media Data: Understanding...,None
1,bd,13rRUwbJCYJ,1900-01-01,"[Binary Codes, Error Correction Codes, Error C...",Code Consistent Hashing Based on Information-T...,Learning based hashing techniques have attract...
2,bd,13rRUzphDs2,1900-01-01,"[Visualization, Search Engines, Google, Big Da...",Exploration of Image Search Results Quality As...,Image retrieval plays an increasingly importan...
3,bd,13rRUNvyan1,1900-01-01,"[Training, Semantics, Machine Learning, Visual...",Weakly Semi-Supervised Deep Learning for Multi...,"In this paper, we study leveraging both weakly..."


In [31]:
all_articles_from1990 = all_articles_df[all_articles_df.pubDate >= pd.to_datetime("1990-01-01")]

In [32]:
quarterly_keywords = all_articles_from1990[['pubDate', 'keywords']].groupby(pd.Grouper(key='pubDate', freq="Q")).agg(sum) # quarterly. http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases

In [33]:
uarterly_keywords = quarterly_keywords[quarterly_keywords['keywords'] != False]

In [34]:
quarterly_keywords = quarterly_keywords[quarterly_keywords.keywords.str.len() != 0]

In [35]:
def remove_none_in_list(li):
    li = [x for x in li if x != '']
    return li

In [36]:
quarterly_keywords.keywords = quarterly_keywords.keywords.apply(remove_none_in_list)

In [37]:
quarterly_keywords.sort_values(by='pubDate')

,keywords
pubDate,
1990-03-31,[Index Termsshared Data Structures Software Qu...
1990-06-30,[Index Termsinstruction Sets Manchester Datafl...
1990-09-30,[Index Termsperformance Prediction Parallel Co...
1990-12-31,[Index Termsfeedback Tree Saturation Control F...
1991-03-31,[Multidimensional Access Memories Connectivity...
1991-06-30,[Index Termssquare Error Hypercube Multicomput...
1991-09-30,[Index Termsparallel Algorithm Granularity Pro...
1991-12-31,[Index Termscontrol Tags Structural Properties...
1992-03-31,[Index Termshexagon Shaped Cylinder Connected ...


In [38]:
def keywords_relative_frequency(li):
    freq = {}
    
    for i in li:
        if i in freq.keys():
            freq[i] += 1
        else:
            freq[i] = 1
            
    for k, v in freq.items():
        freq[k] = v/len(li)
    
    return freq

In [39]:
quarterly_keywords['relativeFrequency'] = quarterly_keywords.keywords.apply(keywords_relative_frequency)

In [40]:
quarterly_keywords.loc['1995-03-31'].relativeFrequency

{'Telecommunication Signalling': 0.0008547008547008547,
 'Telecommunication Congestion Control': 0.003418803418803419,
 'Protocols': 0.003418803418803419,
 'B ISDN': 0.0017094017094017094,
 'Telecommunication Traffic': 0.009401709401709401,
 'Internetworking': 0.002564102564102564,
 'SS 7 Congestion Control': 0.0008547008547008547,
 'Mass Call In Situations': 0.0008547008547008547,
 'ITU T': 0.0008547008547008547,
 'International Telecommunications Union': 0.0008547008547008547,
 'Telecommunications Standardization Sector': 0.0008547008547008547,
 'CCITT': 0.0008547008547008547,
 'SS 7 Protocol': 0.0008547008547008547,
 'Call Throughput': 0.0008547008547008547,
 'International Option': 0.0008547008547008547,
 'Threshold Settings': 0.0008547008547008547,
 'Delays': 0.005128205128205128,
 'National Option': 0.0008547008547008547,
 'Congestion Priorities': 0.0008547008547008547,
 'Application Level Overload Procedures': 0.0008547008547008547,
 'B ISDN Overload': 0.0008547008547008547,
 'W

In [41]:
quarterly_keywords.loc['1995-09-30'].relativeFrequency

{'Telecommunication Traffic': 0.012426900584795321,
 'Internet': 0.0021929824561403508,
 'Computer Network Management': 0.0021929824561403508,
 'Packet Switching': 0.007309941520467836,
 'Multi Access Systems': 0.0043859649122807015,
 'Transport Protocols': 0.0021929824561403508,
 'Resource Management Models': 0.0007309941520467836,
 'Packet Networks': 0.0007309941520467836,
 'Hierarchical Link Sharing': 0.0007309941520467836,
 'Multiple Agencies': 0.0007309941520467836,
 'Protocol Families': 0.0007309941520467836,
 'Traffic Types': 0.0007309941520467836,
 'Real Time Services': 0.0007309941520467836,
 'Gateway': 0.0007309941520467836,
 'Emerging Applications': 0.0007309941520467836,
 'Resource Management': 0.0021929824561403508,
 'Communication System Traffic Control': 0.0029239766081871343,
 'Bandwidth': 0.0007309941520467836,
 'Protocols': 0.0029239766081871343,
 'Scheduling Algorithm': 0.003654970760233918,
 'Jacobian Matrices': 0.0007309941520467836,
 'Traffic Control': 0.005847953

In [42]:
d = quarterly_keywords.loc['2018-03-31'].relativeFrequency

In [43]:
d

{'Feature Extraction': 0.0071315372424722665,
 'Image Reconstruction': 0.000792393026941363,
 'Probabilistic Logic': 0.002377179080824089,
 'Shape': 0.0027733755942947703,
 'Big Data': 0.004754358161648178,
 'Data Mining': 0.001584786053882726,
 'Supervised Learning': 0.0003961965134706815,
 'Sparse Autoencoder': 0.0003961965134706815,
 'Sparse Hyperparameter': 0.0003961965134706815,
 'Feature Extractor': 0.0003961965134706815,
 'Generative Model': 0.0003961965134706815,
 'Security': 0.002377179080824089,
 'Malware': 0.0019809825673534074,
 'Correlation': 0.004358161648177496,
 'Databases': 0.0035657686212361333,
 'Cloud Computing': 0.02377179080824089,
 'Virtualized Infrastructure': 0.0003961965134706815,
 'Virtualization Security': 0.0003961965134706815,
 'Cloud Security': 0.0003961965134706815,
 'Malware Detection': 0.0003961965134706815,
 'Rootkit Detection': 0.0003961965134706815,
 'Security Analytics': 0.0003961965134706815,
 'Event Correlation': 0.0003961965134706815,
 'Logistic

In [44]:
sorted_by_value = sorted(d.items(), key=lambda x: x[1])

In [46]:
#?sorted ##이거 뭐지?

In [47]:
for i in d.items():
    print(i[1])

0.0071315372424722665
0.000792393026941363
0.002377179080824089
0.0027733755942947703
0.004754358161648178
0.001584786053882726
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.002377179080824089
0.0019809825673534074
0.004358161648177496
0.0035657686212361333
0.02377179080824089
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0011885895404120444
0.0027733755942947703
0.0035657686212361333
0.001584786053882726
0.000792393026941363
0.0003961965134706815
0.003169572107765452
0.001584786053882726
0.0003961965134706815
0.0011885895404120444
0.000792393026941363
0.000792393026941363
0.0003961965134706815
0.0003961965134706815
0.000792393026941363
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.0003961965134706815
0.000792393026941363
0.0003961965134706815
0.00

In [48]:
d.items()

dict_items([('Feature Extraction', 0.0071315372424722665), ('Image Reconstruction', 0.000792393026941363), ('Probabilistic Logic', 0.002377179080824089), ('Shape', 0.0027733755942947703), ('Big Data', 0.004754358161648178), ('Data Mining', 0.001584786053882726), ('Supervised Learning', 0.0003961965134706815), ('Sparse Autoencoder', 0.0003961965134706815), ('Sparse Hyperparameter', 0.0003961965134706815), ('Feature Extractor', 0.0003961965134706815), ('Generative Model', 0.0003961965134706815), ('Security', 0.002377179080824089), ('Malware', 0.0019809825673534074), ('Correlation', 0.004358161648177496), ('Databases', 0.0035657686212361333), ('Cloud Computing', 0.02377179080824089), ('Virtualized Infrastructure', 0.0003961965134706815), ('Virtualization Security', 0.0003961965134706815), ('Cloud Security', 0.0003961965134706815), ('Malware Detection', 0.0003961965134706815), ('Rootkit Detection', 0.0003961965134706815), ('Security Analytics', 0.0003961965134706815), ('Event Correlation',

In [49]:
sorted_by_value

[('Supervised Learning', 0.0003961965134706815),
 ('Sparse Autoencoder', 0.0003961965134706815),
 ('Sparse Hyperparameter', 0.0003961965134706815),
 ('Feature Extractor', 0.0003961965134706815),
 ('Generative Model', 0.0003961965134706815),
 ('Virtualized Infrastructure', 0.0003961965134706815),
 ('Virtualization Security', 0.0003961965134706815),
 ('Cloud Security', 0.0003961965134706815),
 ('Malware Detection', 0.0003961965134706815),
 ('Rootkit Detection', 0.0003961965134706815),
 ('Security Analytics', 0.0003961965134706815),
 ('Event Correlation', 0.0003961965134706815),
 ('Logistic Regression', 0.0003961965134706815),
 ('Belief Propagation', 0.0003961965134706815),
 ('Sparse Linear Systems Of Equations', 0.0003961965134706815),
 ('Memory I O Complexity', 0.0003961965134706815),
 ('Multimedia Communication', 0.0003961965134706815),
 ('Distributed Databases', 0.0003961965134706815),
 ('Search Problems', 0.0003961965134706815),
 ('In Memory Processing', 0.0003961965134706815),
 ('Mu

##### articleId로 id, keywords, pubDate, title, year 찾기

In [127]:
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                      json={
                          "variables":{"articleId":"13rRUwwslBG"},
                          "query":"query ($articleId: String!) { articleById(articleId: $articleId) { id keywords pubDate title year } }"
                      })

In [128]:
body = resp.json()

In [129]:
print(json.dumps(body, indent=4))

{
    "data": {
        "articleById": {
            "id": "13rRUwwslBG",
            "keywords": [
                "Nonvolatile Memory",
                "Random Access Memory",
                "Encryption",
                "Computer Architecture",
                "System On Chip",
                "Oblivious RAM",
                "Non Volatile Memory",
                "Memory Security"
            ],
            "pubDate": "2018-07-01",
            "title": "LEO: Low Overhead Encryption ORAM for Non-Volatile Memories",
            "year": "2018"
        }
    }
}
